建立模型时，可能会遇到正负样本比例极度不均衡的情况。例如，建立信用违约模型时，违约样本的比例远小于不违约样本的比例，此时模型会花更多精力去拟合不违约样本，但实际上找出违约样本更为重要。这会导致模型可能在训练集上表现良好，但测试时表现不佳。为了改善样本比例不均衡的问题，可以使用过采样和欠采样的方法。
> 场景假设：建立信用违约模型时，样本数据中有1000个不违约样本和100个违约样本

## 一、过采样
### 1、过采样的原理
#### （1）随机过采样
随机过采样是从100个违约样本中随机抽取旧样本作为一个新样本，共反复抽取900次，然后和原来的100个旧样本组合成新的1000个违约样本，和1000个不违约样本一起构成新的训练集。但因为随机过采样重复地选取了违约样本，所以有可能造成对违约样本的过拟合。
#### （2）SMOTE法过采样
SMOTE法过采样即合成少数类过采样技术，它是一种针对随机过采样容易导致过拟合问题的改进方案。
> 假设对少数类进行4倍过采样
![SMOTE过采样](image/SMOTE过采样.png)

### 2、过采样的代码实现

In [2]:
# 读取数据
import pandas as pd
data = pd.read_excel('datasets/信用卡数据.xlsx')
data.head()

,编号,年龄,负债比率,月收入,贷款数量,家属人数,分类
0,1,29,0.22,7800,1,3,0
1,2,52,0.46,4650,1,0,0
2,3,28,0.10,3000,0,0,0
3,4,29,0.20,5916,0,0,0
4,5,27,1.28,1300,0,0,1


In [3]:
# 提取特征变量和目标变量
x = data.drop(columns='分类')
y = data['分类']
# 用collections库中的Counter()函数对目标变量进行计数
from collections import Counter
Counter(y)

Counter({0: 1000, 1: 100})

结果显示：不违约样本数为1000，远远大于违约样本数100

#### （1）随机过采样
安装imlearn库，imblearn库是Python中专门用来处理数据不均衡问题的工具库

In [5]:
!pip install  imbalanced-learn

     |████████████████████████████████| 199 kB 906 kB/s eta 0:00:01


In [8]:
from imblearn.over_sampling import RandomOverSampler # 从imblearn库中引入用来进行随机过采样的RandomOverSampler()函数
ros = RandomOverSampler(random_state=0) # 设置RandomOverSampler()函数的参数random_state为0（此数字没有特殊含义，可以换成其他数字），使得每次代码运行的结果保持一致
x_oversampled, y_oversampled = ros.fit_resample(x, y) # 使用原始数据的特征变量和目标变量生成过采样数据集，并分别赋给变量X_oversampled和y_oversampled

In [9]:
# 检验随机过采样的效果
Counter(y_oversampled)

Counter({0: 1000, 1: 1000})

In [20]:
# 结果的2000就是违约样本数1000和不违约样本数1000之和，随机过采样后特征变量的数据也随之增多
x_oversampled.shape

(2000, 6)

#### （2）SMOTE法过采样

In [11]:
from imblearn.over_sampling import SMOTE # 从imblearn库中引入用于进行SMOTE法过采样的SMOTE()函数
smote = SMOTE(random_state=0) # 设置SMOTE()函数的参数random_state为0（此数字没有特殊含义，可以换成其他数字），使得每次代码运行的结果保持一致
x_smotsampled, y_smotsampled = smote.fit_resample(x, y) # 使用原始数据的特征变量和目标变量生成过采样数据集并分别赋给变量x_smotesampled和y_smotesampled

In [13]:
# 检验随机过采样的效果
Counter(y_oversampled)

Counter({0: 1000, 1: 1000})

## 二、欠采样
### 1、欠采样的原理
欠采样是从1000个不违约样本中随机选取100个样本，和100个违约样本一起构成新的训练集。欠采样抛弃了大部分不违约样本，在搭建模型时有可能产生欠拟合。

### 2、欠采样的代码实现

In [18]:
from imblearn.under_sampling import RandomUnderSampler # 从imblearn库中引入用于进行随机欠采样的RandomUnderSampler()函数
rus = RandomUnderSampler(random_state=0) # 设置RandomUnderSampler()函数的参数random_state为0（此数字没有特殊含义，可以换成其他数字），使得每次代码运行的结果保持一致
x_undersampled, y_undersampled = rus.fit_resample(x, y) # 使用原始数据的特征变量和目标变量生成欠采样数据集并分别赋给变量x_undersampled和y_undersampled

In [19]:
# 用Counter()函数检验一下随机欠采样的效果
Counter(y_undersampled)

Counter({0: 100, 1: 100})

不违约样本数从1000下降至100，与违约样本数相同，证明随机欠采样有效

In [21]:
# 结果的200就是违约样本数100和不违约样本数100之和，可以看到，随机欠采样后特征变量的数据也随之减少
x_undersampled.shape

(200, 6)

## 总结

在实战中处理样本不均衡问题时，如果样本数据量不大，通常使用过采样，因为这样能更好地利用数据，不会像欠采样那样很多数据都没有使用到；如果数据量充足，则过采样和欠采样都可以考虑使用。